# 5. Get_Predict_Result_From_Redis

## Run name

In [1]:
import time

project_name = 'Google_LandMark_Rec2019'
step_name = '5_Get_Predict_Result_From_Redis'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: Google_LandMark_Rec2019_5_Get_Predict_Result_From_Redis_20190531_232822


## Important params

In [2]:
import multiprocessing

cpu_amount = multiprocessing.cpu_count()
print('cpu_amount: ', cpu_amount)

topn = 100
print('topn:\t', topn)

cpu_amount:  4
topn:	 100


## Import PKGs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os
import sys
import gc
import math
import shutil
import zipfile
import pickle
import h5py
import json

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [4]:
from feature import load_feature
from utility import pickle_load

## Project folders

In [5]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
val_folder = os.path.join(input_folder, 'data_val')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_csv = os.path.join(input_folder, 'sample_submission.csv')
recognition_sample_submission_csv = os.path.join(input_folder, 'recognition_sample_submission.csv')
retrieval_sample_submission_csv = os.path.join(input_folder, 'retrieval_sample_submission.csv')
data_test_00_filenames_pkl = os.path.join(feature_folder, 'feature_VGG19_data_test_00_171023_filenames.pkl')
stage2_test_csv = os.path.join(input_folder, 'test.csv')

## Redis client

In [6]:
import redis
redis_cli = redis.Redis(host='18.188.192.187', port=6379, password='elect')

## Get result from redis

In [7]:
def get_predict_result(redis_cli, end=10, start=0):
    pred_result_ser_arr = redis_cli.lrange('test_pred_result', start, end)
    pred_result_arr = [json.loads(pred_result_ser) for pred_result_ser in pred_result_ser_arr if pred_result_ser]
    return pred_result_arr

# pred_result_arr = get_predict_result(redis_cli, 10)
# print('len(pred_result_arr):', len(pred_result_arr))

In [8]:
# for item_pred_result in pred_result_arr[0: 10]:
#     print('%10d' % item_pred_result['id'], '%10d' % item_pred_result['top1_pred'], '%10d' % item_pred_result['weighted_top1_pred'])

## Load val feature

In [9]:
# x_val, y_val, idx_val = load_feature(feature_folder, 'test', 'VGG19', '171023', 1)
# display(x_val.shape)
# display(y_val.shape)
# display(idx_val.shape)

In [10]:
# debug_row = len(pred_result_arr)
# amount = debug_row
# top1_count = 0
# weighted_top1_count = 0
# topn_count = 0
# for item_idx, item_y, item_pred_result in zip(idx_val[:debug_row], y_val[:debug_row], pred_result_arr[:debug_row]):
#     top1_status = 0
#     weighted_top1_status = 0
#     topn_status = 0
#     if item_y == item_pred_result['top1_pred']:
#         top1_count += 1
#         top1_status = 1
#     if item_y == item_pred_result['weighted_top1_pred']:
#         weighted_top1_count += 1
#         weighted_top1_status = 1
#     if isinstance(item_pred_result['topn_pred_arr'], list) and item_y in item_pred_result['topn_pred_arr']:
#         topn_count += 1
#         topn_status = 1
#     if item_idx <= 20 or item_idx >= (debug_row-20):
#         print('%10d' % item_idx, '%10d' % item_y, '%10d' % item_pred_result['id'], '%10d' % item_pred_result['top1_pred'], '%10d' % item_pred_result['weighted_top1_pred'], '%d %d %d' % (top1_status, weighted_top1_status, topn_status))

# print('*' * 80)
# print(top1_count, '%.2f' % (top1_count/amount))
# print(weighted_top1_count, '%.2f' % (weighted_top1_count/amount))
# print(topn_count, '%.2f' % (topn_count/amount))

In [11]:
# acc10000 = int(topn_count/amount*10000)
# if acc10000 == 10000:
#     run_name_acc = '%s_%05d' % (run_name, acc10000)
# else:
#     run_name_acc = '%s_%04d' % (run_name, acc10000)
# print(run_name_acc)

## Load data_test_00 predict result

In [12]:
stage2_test_df = pd.read_csv(stage2_test_csv)
display(stage2_test_df.shape, stage2_test_df.head(2))

(117577, 1)

,id
0,00016575233bc956
1,0001aadbcd8cb923


In [13]:
recognition_sample_submission_df = stage2_test_df.copy('K')
recognition_sample_submission_df['landmarks'] = ''
display(recognition_sample_submission_df.shape, recognition_sample_submission_df.head())

(117577, 2)

,id,landmarks
0,00016575233bc956,
1,0001aadbcd8cb923,
2,0002c06b2440a5f9,
3,0002eb1ee5a5a6b2,
4,000594dad986513e,


In [14]:
recognition_sample_submission_df['landmarks'] = ''
recognition_sample_submission_df.set_index(['id'], inplace=True)
display(recognition_sample_submission_df.shape, recognition_sample_submission_df.head())

(117577, 1)

,landmarks
id,
00016575233bc956,
0001aadbcd8cb923,
0002c06b2440a5f9,
0002eb1ee5a5a6b2,
000594dad986513e,


In [15]:
retrieval_sample_submission_df = stage2_test_df.copy('K')
retrieval_sample_submission_df['images'] = ''
display(retrieval_sample_submission_df.shape, retrieval_sample_submission_df.head())

(117577, 2)

,id,images
0,00016575233bc956,
1,0001aadbcd8cb923,
2,0002c06b2440a5f9,
3,0002eb1ee5a5a6b2,
4,000594dad986513e,


In [16]:
retrieval_sample_submission_df['images'] = ''
retrieval_sample_submission_df.set_index(['id'], inplace=True)
display(retrieval_sample_submission_df.shape, retrieval_sample_submission_df.head())

(117577, 1)

,images
id,
00016575233bc956,
0001aadbcd8cb923,
0002c06b2440a5f9,
0002eb1ee5a5a6b2,
000594dad986513e,


In [17]:
data_test_00_filenames = pickle_load(data_test_00_filenames_pkl)
print(len(data_test_00_filenames), data_test_00_filenames[:5])

112749 ['0/000088da12d664db.jpg', '0/0001623c6d808702.jpg', '0/0001bbb682d45002.jpg', '0/0002362830cfe3a3.jpg', '0/000270c9100de789.jpg']


In [18]:
image_ids = [filename[2: -4] for filename in data_test_00_filenames]
print(len(image_ids), image_ids[:5])

112749 ['000088da12d664db', '0001623c6d808702', '0001bbb682d45002', '0002362830cfe3a3', '000270c9100de789']


In [19]:
pred_result_arr = get_predict_result(redis_cli, -1, 0)
print('len(pred_result_arr):', len(pred_result_arr))

len(pred_result_arr): 3903


In [20]:
def get_uniform_topn(y_pred, y_proba, topn, ndigits=3):
    y_pred_reverse = y_pred[::-1]
    y_proba_reverse = y_proba[::-1]
    y_pred_topn = y_pred_reverse[:topn]
    y_proba_topn = y_proba_reverse[:topn]
    y_proba_topn_sum = sum(y_proba_topn)
    y_pred_topn = [item for item in y_pred_topn]
    y_proba_topn = [round(item/y_proba_topn_sum, ndigits) for item in y_proba_topn]
    return y_pred_topn, y_proba_topn

# demo_y_pred = list(range(10))
# demo_y_proba = [item for item in list(range(10))]
# demo_y_pred_topn, demo_y_proba_topn = get_uniform_topn(demo_y_pred, demo_y_proba, 5)

In [21]:
# print(sum(demo_y_proba_topn))

In [22]:
print('len(pred_result_arr):', len(pred_result_arr))

len(pred_result_arr): 3903


In [23]:
for item_pred_result in pred_result_arr:
    test_data_idx = item_pred_result['id']
    topn_filename_arr = item_pred_result['topn_filename_arr']
    image_id = image_ids[test_data_idx]
    item_y_pred_topn, item_y_proba_topn = get_uniform_topn(item_pred_result['topn_pred_arr'], item_pred_result['topn_proba_arr'], topn)
    # recognition
#     print(item_pred_result['top1_pred'])
    recognition_sample_submission_df.at[image_id, 'landmarks'] = '%s %.2f' % (item_pred_result['top1_pred'], item_y_proba_topn[0])
    # retrieval
    pred_filename_arr = topn_filename_arr[-topn:]
    pred_filename_arr.reverse()
    retrieval_sample_submission_df.at[image_id, 'images'] = ' '.join([filename[-20:-4] for filename in pred_filename_arr])
    if test_data_idx < 100:
        print('%7d' % test_data_idx, image_id, item_pred_result['top1_pred'], item_pred_result['topn_pred_arr'][-1], item_y_pred_topn[0], item_y_proba_topn[0], item_pred_result['topn_proba_arr'][-1], len(pred_filename_arr))

      0 000088da12d664db 65218 65218 65218 0.01 0.971 100
      1 0001623c6d808702 138982 138982 138982 0.01 0.992 100
      2 0001bbb682d45002 174821 174821 174821 0.01 0.994 100
      3 0002362830cfe3a3 42297 42297 42297 0.01 0.998 100
      4 000270c9100de789 145325 145325 145325 0.01 0.999 100
      5 0002b0fab5d3ccc4 198778 198778 198778 0.01 0.993 100
      6 000506dc6ab3a40e 138719 138719 138719 0.01 0.995 100
      7 0005292fc4b005a3 96663 96663 96663 0.01 0.999 100
      8 0005456a82264bc8 82137 82137 82137 0.01 0.988 100
      9 00055cf2bfb5594a 164179 164179 164179 0.01 1.0 100
     10 000664eed4a70821 129024 129024 129024 0.01 0.999 100
     11 0006aea5b6f4eaaa 145097 145097 145097 0.01 0.997 100
     12 0006bbfa00dd6c0f 195843 195843 195843 0.01 0.997 100
     13 0008aee1c0abed9d 145015 145015 145015 0.01 0.998 100
     14 0008de5f3c25d563 24607 24607 24607 0.01 0.996 100
     15 00094466c9f054f4 154960 154960 154960 0.01 0.986 100
     16 0009f09a69405693 88596 88596 8859

## Export submission

In [24]:
recognition_sample_submission_df.reset_index(drop=False, inplace=True)
retrieval_sample_submission_df.reset_index(drop=False, inplace=True)

In [25]:
recognition_sample_submission_df_concreate = recognition_sample_submission_df[recognition_sample_submission_df['landmarks'] != '']
display(recognition_sample_submission_df_concreate.shape, recognition_sample_submission_df_concreate.head())
retrieval_sample_submission_df_concreate = retrieval_sample_submission_df[retrieval_sample_submission_df['images'] != '']
display(retrieval_sample_submission_df_concreate.shape, retrieval_sample_submission_df_concreate.head())

(3903, 2)

,id,landmarks
117577,000088da12d664db,65218 0.01
117578,0001623c6d808702,138982 0.01
117579,0001bbb682d45002,174821 0.01
117580,0002362830cfe3a3,42297 0.01
117581,000270c9100de789,145325 0.01


(3903, 2)

,id,images
117577,000088da12d664db,afa533419f7ab5ac 1e393b7ab22eb960 f51620d23acd...
117578,0001623c6d808702,df0e1f887e3a7c03 f8a65bbb71c96da0 75b39c7111cb...
117579,0001bbb682d45002,301c36bdf8878428 579602e71283374c 682912c05f73...
117580,0002362830cfe3a3,26e3756c1eb9ce06 15ef6bfd1d98435f bbdebf3d55d1...
117581,000270c9100de789,ea03cd7d0f68a60d 4fadce647ef3e0cb c2b4fbbc07ef...


In [26]:
display(recognition_sample_submission_df.shape, recognition_sample_submission_df.head())
display(retrieval_sample_submission_df.shape, retrieval_sample_submission_df.head())

(121480, 2)

,id,landmarks
0,00016575233bc956,
1,0001aadbcd8cb923,
2,0002c06b2440a5f9,
3,0002eb1ee5a5a6b2,
4,000594dad986513e,


(121480, 2)

,id,images
0,00016575233bc956,
1,0001aadbcd8cb923,
2,0002c06b2440a5f9,
3,0002eb1ee5a5a6b2,
4,000594dad986513e,


In [27]:
recognition_submission_file = os.path.join(output_folder, '%s_recognition.csv' % run_name)
print(recognition_submission_file)
recognition_sample_submission_df.to_csv(recognition_submission_file, index=False)

D:\Kaggle\landmark-recognition-2019\output\Google_LandMark_Rec2019_5_Get_Predict_Result_From_Redis_20190531_232822_recognition.csv


In [30]:
retrieval_submission_file = os.path.join(output_folder, '%s_retrieval.csv' % run_name)
print(retrieval_submission_file)
retrieval_sample_submission_df.to_csv(retrieval_submission_file, index=False)

D:\Kaggle\landmark-recognition-2019\output\Google_LandMark_Rec2019_5_Get_Predict_Result_From_Redis_20190531_232822_retrieval.csv


In [29]:
print('Time elapsed: %.1fs' % (time.time() - t0))
print(run_name)

Time elapsed: 1615.5s
Google_LandMark_Rec2019_5_Get_Predict_Result_From_Redis_20190531_232822
